# RSA as (optimal?) scene compression?

RSA models have been succesful at predicting human pragmatic judgments. That is, given a `scene` including 
```
obj1:<blue square>

obj2:<blue circle>

obj3:<green square>

```
listeners can succesfully identify `obj1` simply by `blue` (reasoning that it would have been more informative to refer to `obj3` as `green`.

That's great, but RSA actually tells us more. RSA says that we can also unambiguously refer to `obj1` as `square` and `obj2` as `circle`!

Imagine that instead of using words we mapped the adjectives to binar codes:
```
blue -> 00

green -> 01

square -> 10

circle -> 11
```
A **literal** description of the scene might look like:

```
"blue square, blue circle, green square" -> 001000110110
```

which requires 12 bits.

An RSA L2 speaker tells us we could refer to the same objects using:

```
blue, circle, green" -> 001101
```

which requires only 6 bits, reducing the total scene description size by half.

### This notebook explores the idea that 
0. RSA pragmatic listeners characterize optimal compression schemes of scenes
0. That the difference between the "optimal" copression scheme and the literal description represent the (latent) pragmatic content available in a scene.

### Todo
* what about when there is "partial" pragmatic information (e.g. `<blue square, green square, blue square>`). E.g. we can compress `<green square>` to `green`.

### Note
* matrix computations can disrupt orders below.


In [1]:
from collections import defaultdict
import numpy as np

## Helpers

In [2]:
def norm(arr):
    return arr / np.sum(arr) if np.sum(arr) != 0 else arr

def L(m):
    return np.apply_along_axis(norm, 1, m)

def S(m):
    return np.apply_along_axis(norm, 0, m)

def rsa(m, depth):
    for i in range(depth, 0, -1):
        if i == 0:
            return m
        if i % 2 == 0:
            m = S(m)
        else:
            m = L(m)
    return m

def equals_max(arr):
    return [x == np.max(arr) and x != 0 for x in arr]

def round_matrix(m):
    return np.apply_along_axis(equals_max, 1, m).astype(int)

def can_be_compressed(m):
    return all([x == 1 for x in np.apply_along_axis(np.sum, 1, m)])

def compress_matrix(m):
    return np.unique(m, axis=0)

# Simulation 1

In [3]:
class Object:
    def __init__(self, color, shape, language):
        self.color = color
        self.shape = shape
        self.color_bin = language[self.color]
        self.shape_bin = language[self.shape]
    
    def get_text_descr(self):
        return self.color + ' ' + self.shape
    
    def get_bin_color(self):
        return self.color_bin
    
    def get_bin_shape(self):
        return self.shape_bin
    
    def get_bin_descr(self):
        return self.get_bin_color() + self.get_bin_shape()

class Scene:
    def __init__(self, object1, object2, object3):
        self.object1 = object1
        self.object2 = object2
        self.object3 = object3
        self.scene = [self.object1, self.object2, self.object3]
        self.literal_matrix = self.build_literal_matrix(*self.scene)        
    
    def get_text_descr(self):
        return [self.object1.get_text_descr(), self.object2.get_text_descr(), self.object3.get_text_descr()]
    
    def get_bin_descr(self):
        return self.object1.get_bin_descr() + self.object2.get_bin_descr() + self.object3.get_bin_descr()
    
    def get_scene(self):
        return self.scene
    
    def build_literal_matrix(self, obj1, obj2, obj3):
        return np.array([
            np.array(list(map(is_blue, self.get_scene()))),
            np.array(list(map(is_red, self.get_scene()))),
            np.array(list(map(is_square, self.get_scene()))),
            np.array(list(map(is_circle, self.get_scene())))])
    
    def get_literal_matrix(self):
        return self.literal_matrix

## World 1 - colors/shapes

In [4]:
# Elements
words2bin = {
    'blue' : '00',
    'green' : '10',
    'square' : '01',
    'circle' : '11'
}

bin2words = {
    '00' : 'blue',
    '10' : 'green',
    '01' : 'square',
    '11' : 'circle'
}

In [5]:
def is_dim(dim):
    def color_lookup(obj):
        if dim == 'square' or dim == 'circle':
            return 1 if obj.get_bin_shape() == words2bin[dim] else 0
        else:
            return 1 if obj.get_bin_color() == words2bin[dim] else 0
    return color_lookup

is_blue = is_dim('blue')
is_red = is_dim('green')
is_square = is_dim('square')
is_circle = is_dim('circle')

In [6]:
def matrix2code(m, rows):
    """Convert a matrix into (possible) scene code(s).
    
    Parameters
    ----------
    rows: string
        bin descriptions.
    
    Returns
    -------
    String
        New scene description.
        
    """
    d = defaultdict(list)
    for i, r in enumerate(rows):
        for j, item in enumerate(m[i]):
            if item == 1 and not j in d:
                d[j] = r
    return list(d.values())

def code2descr(arr, bin_lang):
    return [bin_lang[binar] for binar in arr]

### Example 1 - Frank & Goodman (2012) implicature

[`blue square`] [`blue circle`] [`green square`]

In [7]:
# Objects
obj1 = Object('blue', 'circle', words2bin)
obj2 = Object('blue', 'square', words2bin)
obj3 = Object('green', 'square', words2bin)
scene = Scene(obj1, obj2, obj3)

In [8]:
print("Literal scene description:\t{}".format(scene.get_text_descr()))

Literal scene description:	['blue circle', 'blue square', 'green square']


Run RSA so we get L1 listener matrix

In [10]:
m = rsa(scene.get_literal_matrix(), 3) # 1 = L0, 2 = S1, 2 = L1
print("L1 matrix:\n{}".format(m))
m_decision = round_matrix(m)
print()
print("L1 decision matrix:\n{}".format(m_decision))

L1 matrix:
[[ 0.4  0.6  0. ]
 [ 0.   0.   1. ]
 [ 0.   0.6  0.4]
 [ 1.   0.   0. ]]

L1 decision matrix:
[[0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]]


In [11]:
compressed_code = ''.join(matrix2code(m_decision, bin2words.keys()))
print("Literal scene text descr:\t{}".format(scene.get_text_descr()))
print("compressed scene descr:\t\t{}".format(' '.join(code2descr(matrix2code(m_decision, bin2words.keys()), bin2words))))
print("Literal scene binary descr:\t{}".format(scene.get_bin_descr()))
print("Compressed scene code:\t\t{}".format(compressed_code))
print("compression factor:\t\t{}".format(len(compressed_code)/len(scene.get_bin_descr())))

Literal scene text descr:	['blue circle', 'blue square', 'green square']
compressed scene descr:		blue green circle
Literal scene binary descr:	001100011001
Compressed scene code:		001011
compression factor:		0.5


### Example 2 - Frank & Goodman (2012) implicature

[`green square`] [`green circle`] [`blue circle`]

In [12]:
# Objects
obj1 = Object('green', 'square', words2bin)
obj2 = Object('green', 'circle', words2bin)
obj3 = Object('blue', 'square', words2bin)
scene1 = Scene(obj1, obj2, obj3)

In [13]:
m = rsa(scene.get_literal_matrix(), 3) # 1 = L0, 2 = S1, 2 = L1
print("L1 matrix:\n{}".format(m))
m_decision = round_matrix(m)
print()
print("L1 decision matrix:\n{}".format(m_decision))

L1 matrix:
[[ 0.4  0.6  0. ]
 [ 0.   0.   1. ]
 [ 0.   0.6  0.4]
 [ 1.   0.   0. ]]

L1 decision matrix:
[[0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]]


In [14]:
compressed_code = ''.join(matrix2code(m_decision, bin2words.keys()))
print("Literal scene text descr:\t{}".format(scene.get_text_descr()))
print("compressed scene descr:\t\t{}".format(' '.join(code2descr(matrix2code(m_decision, bin2words.keys()), bin2words))))
print("Literal scene binary descr:\t{}".format(scene.get_bin_descr()))
print("Compressed scene code:\t\t{}".format(compressed_code))
print("compression factor:\t\t{}".format(len(compressed_code)/len(scene.get_bin_descr())))

Literal scene text descr:	['blue circle', 'blue square', 'green square']
compressed scene descr:		blue green circle
Literal scene binary descr:	001100011001
Compressed scene code:		001011
compression factor:		0.5


# Simulation 2

### Frank & Goodman Hat, Glasses, Mustache people...

In [15]:
# Elements
words2bin = {
    'h' : '00',
    'g' : '10',
    'm' : '01',
    'n': '11',
    '':''
}

bin2words = {
    '00' : 'h',
    '10' : 'g',
    '01' : 'm',
    '11' : 'n'
}

In [16]:
class Object:
    def __init__(self, hat, glasses, mustache, lang):
        self.hat = 'h' if hat else ''
        self.glasses = 'g' if glasses else ''
        self.mustache = 'm' if mustache else ''
        self.none = 'n' if hat | glasses | mustache == False else ''
        self.lang = lang
    
    def has_glasses(self):
        return self.glasses == 'g'
    
    def has_hat(self):
        return self.hat == 'h'
    
    def has_mustache(self):
        return self.mustache == 'm'
    
    def get_text_descr(self):
        return ''.join([self.hat, self.glasses, self.mustache])
    
    def get_bin_descr(self):
        return ''.join([self.lang[self.hat], self.lang[self.glasses], self.lang[self.mustache], self.lang[self.none]])

class Scene:
    def __init__(self, object1, object2, object3):
        self.object1 = object1
        self.object2 = object2
        self.object3 = object3
        self.scene = [self.object1, self.object2, self.object3]
        self.literal_matrix = self.build_literal_matrix(*self.scene)        
    
    def get_text_descr(self):
        return [self.object1.get_text_descr(), self.object2.get_text_descr(), self.object3.get_text_descr()]
    
    def get_bin_descr(self):
        return self.object1.get_bin_descr() + self.object2.get_bin_descr() + self.object3.get_bin_descr()
    
    def get_scene(self):
        return self.scene
    
    def build_literal_matrix(self, obj1, obj2, obj3):
        return np.array([
            np.array(list(map(has_hat, self.get_scene()))),
            np.array(list(map(has_glasses, self.get_scene()))),
            np.array(list(map(has_mustache, self.get_scene()))),
            np.array(list(map(has_none, self.get_scene())))])
    
    def get_literal_matrix(self):
        return self.literal_matrix

In [17]:
def is_dim(dim):
    def dim_lookup(obj):
        if dim == 'hat':
            return obj.has_hat()
        elif dim == 'glasses':
            return obj.has_glasses()
        elif dim == 'mustache':
            return obj.has_mustache()
        elif dim == 'none':
            return not obj.has_hat() | obj.has_glasses() | obj.has_mustache()
    return dim_lookup

has_hat = is_dim('hat')
has_glasses = is_dim('glasses')
has_mustache = is_dim('mustache')
has_none = is_dim('none')

World:

[`none`] [`glasses`] [`hat, glasses`]

In [18]:
obj1 = Object(False, False, False, words2bin)
obj2 = Object(False, True, False, words2bin)
obj3 = Object(True, True, False, words2bin)
scene = Scene(obj1, obj2, obj3)

In [19]:
print("Literal scene description text:\t\t{}".format(scene.get_text_descr()))
print("Literal scene description text bin:\t{}".format(scene.get_bin_descr()))

Literal scene description text:		['', 'g', 'hg']
Literal scene description text bin:	11100010


In [20]:
m = rsa(scene.get_literal_matrix(), 3) # 1 = L0, 2 = S1, 2 = L1
print("L1 matrix:\n{}".format(m))
m_decision = round_matrix(m)
print()
print("L1 decision matrix:\n{}".format(m_decision))

L1 matrix:
[[ 0.    0.    1.  ]
 [ 0.    0.75  0.25]
 [ 0.    0.    0.  ]
 [ 1.    0.    0.  ]]

L1 decision matrix:
[[0 0 1]
 [0 1 0]
 [0 0 0]
 [1 0 0]]


In [21]:
compressed_code = ''.join(matrix2code(m_decision, bin2words.keys()))
print("Literal scene text descr:\t{}".format(scene.get_text_descr()))
print("compressed scene descr:\t\t{}".format(' '.join(code2descr(matrix2code(m_decision, bin2words.keys()), bin2words))))
print("Literal scene binary descr:\t{}".format(scene.get_bin_descr()))
print("Compressed scene code:\t\t{}".format(compressed_code))
print("compression factor:\t\t{}".format(len(compressed_code)/len(scene.get_bin_descr())))

Literal scene text descr:	['', 'g', 'hg']
compressed scene descr:		h g n
Literal scene binary descr:	11100010
Compressed scene code:		001011
compression factor:		0.75
